In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
conf = SparkConf().setAppName("App")
conf = (conf.set('spark.driver.memory', '60G').set("spark.yarn.executor.memoryOverhead", '4096'))
sc =SparkContext(conf=conf)
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [3]:
#reading in sticker type data
feature_data = sqlContext.read.parquet("gs://ds-url-catag/Stickers/stick_statistics/derived_features/")


In [3]:
#convert pyspark to pandas dataframe
import pandas as pd
import pyarrow.parquet as pq
feature_data_df  = feature_data.toPandas()

In [4]:
#checking size of dataframe
print 'Size of the dataframe: {}'.format(feature_data_df.shape)

Size of the dataframe: (7495040, 9)


In [5]:
#logarithmic transformation of two different variables
import numpy as np
feature_data_df['log_noofdays']=np.log10(1+feature_data_df.numofdays)
feature_data_df['log_avg_con_days']=np.log10(1+feature_data_df.avg_con_days)
feature_data_df['log_sticker_packs_sent']=np.log10(1+feature_data_df.sticker_packs_sent)
feature_data_df['log_distinct_sticker_packs_sent']=np.log10(1+feature_data_df.distinct_sticker_packs_sent)
feature_data_df['log_sum_paid']=np.log10(1+feature_data_df.sum_paid)
feature_data_df['log_sum_free']=np.log10(1+feature_data_df.sum_free)
feature_data_df['log_sum_subs']=np.log10(1+feature_data_df.sum_subs)
feature_data_df['log_sum_discont']=np.log10(1+feature_data_df.sum_discont)


In [ ]:
import time

from sklearn.manifold import TSNE

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature_data_df[['log_noofdays','log_avg_con_days','log_sticker_packs_sent','log_distinct_sticker_packs_sent','log_sum_paid','log_sum_free','log_sum_subs','log_sum_discont']].values)

print 't-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start)

[t-SNE] Computing 121 nearest neighbors...


In [ ]:
#assigning tsne data to the feature dataset 
feature_data_df['tsne_one'] = tsne_results[:,0]
feature_data_df['tsne_two'] = tsne_results[:,1]
#Kmeans cluster 
from sklearn.cluster import KMeans
clusterer = KMeans(n_clusters=9, random_state=10)
cluster_labels = clusterer.fit_predict(feature_data_df[['tsne_one','tsne_two']])

# clusterer.labels_
feature_data_df['clusters'] = clusterer.labels_



In [7]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=9)
model = kmeans.fit(feature_data_df[['pca-one','pca-two','pca-three','pca-four','pca-five','pca-six','pca-seven']])
y_kmeans = kmeans.predict(feature_data_df[['pca-one','pca-two','pca-three','pca-four','pca-five','pca-six','pca-seven']])
centers = kmeans.cluster_centers_

In [8]:
feature_data_df['clusters'] =  kmeans.labels_

In [9]:
original_data = sqlContext.createDataFrame(feature_data_df)

In [10]:
#distributionofclusters
data = original_data.groupby('clusters').agg(countDistinct('user_id_n').alias('cnt'))
total = data.select("cnt").agg({"cnt": "sum"}).collect().pop()['sum(cnt)']
data = data.withColumn("percentage", (data['cnt']/total) * 100)
data.sort(col('percentage').desc()).show(data.count())

+--------+-------+------------------+
|clusters|    cnt|        percentage|
+--------+-------+------------------+
|       2|2190787| 29.22982399026556|
|       7|1616061|21.561739497054052|
|       6| 898722|11.990889975236957|
|       0| 735645| 9.815091046879003|
|       1| 659583| 8.800259905217317|
|       5| 493926| 6.590038211937495|
|       4| 438667| 5.852763961233029|
|       3| 247672| 3.304478695243788|
|       8| 213977|2.8549147169327984|
+--------+-------+------------------+



In [11]:
aggregated_demo_stick = sqlContext.read.parquet("gs://ds-url-catag/clustering/user_profile_data/")

In [15]:
new_data = original_data.join(aggregated_demo_stick,aggregated_demo_stick.user_id_n == original_data.user_id_n).select([original_data.user_id_n,original_data.clusters,original_data.numofdays,original_data.avg_con_days,original_data.sticker_packs_sent,original_data.distinct_sticker_packs_sent,original_data.sum_paid,original_data.sum_free,original_data.sum_subs,original_data.sum_discont]+[aggregated_demo_stick.city,aggregated_demo_stick.Country,aggregated_demo_stick.age_group,aggregated_demo_stick.gender,aggregated_demo_stick.Device_Brand,aggregated_demo_stick.Device_Model,aggregated_demo_stick.OS_version,aggregated_demo_stick.platform])

In [ ]:
# cluster_8.groupby('clusters','numofdays','avg_con_days','sticker_packs_sent','distinct_sticker_packs_sent','sum_paid','sum_free','sum_subs').count().sort(col("clusters").desc()).show(1000)

In [147]:
cluster_1 = new_data.where(col('clusters')==1)

In [148]:
from pyspark.sql.functions import mean, min, max

In [149]:
cluster_1.select([mean('numofdays'), min('numofdays'), max('numofdays')]).show()

+-----------------+--------------+--------------+
|   avg(numofdays)|min(numofdays)|max(numofdays)|
+-----------------+--------------+--------------+
|8.457347912529633|             1|            26|
+-----------------+--------------+--------------+



In [150]:
cluster_1.select([mean('avg_con_days'), min('avg_con_days'), max('avg_con_days')]).show()

+-----------------+-----------------+-----------------+
|avg(avg_con_days)|min(avg_con_days)|max(avg_con_days)|
+-----------------+-----------------+-----------------+
| 1.41172437765587|              0.0|             10.0|
+-----------------+-----------------+-----------------+



In [151]:
cluster_1.select([mean('sticker_packs_sent'), min('sticker_packs_sent'), max('sticker_packs_sent')]).show()

+-----------------------+-----------------------+-----------------------+
|avg(sticker_packs_sent)|min(sticker_packs_sent)|max(sticker_packs_sent)|
+-----------------------+-----------------------+-----------------------+
|     23.352051266939537|                      6|                     76|
+-----------------------+-----------------------+-----------------------+



In [152]:
cluster_1.select([mean('distinct_sticker_packs_sent'), min('distinct_sticker_packs_sent'), max('distinct_sticker_packs_sent')]).show()

+--------------------------------+--------------------------------+--------------------------------+
|avg(distinct_sticker_packs_sent)|min(distinct_sticker_packs_sent)|max(distinct_sticker_packs_sent)|
+--------------------------------+--------------------------------+--------------------------------+
|               6.064450519777776|                               3|                              49|
+--------------------------------+--------------------------------+--------------------------------+



In [153]:
cluster_1.select([mean('sum_paid'), min('sum_paid'), max('sum_paid')]).show()

+-------------------+-------------+-------------+
|      avg(sum_paid)|min(sum_paid)|max(sum_paid)|
+-------------------+-------------+-------------+
|0.00873212230263419|            0|           14|
+-------------------+-------------+-------------+



In [154]:
cluster_1.select([mean('sum_free'), min('sum_free'), max('sum_free')]).show()

+-----------------+-------------+-------------+
|    avg(sum_free)|min(sum_free)|max(sum_free)|
+-----------------+-------------+-------------+
|5.091552573750952|            1|           44|
+-----------------+-------------+-------------+



In [155]:
cluster_1.select([mean('sum_subs'), min('sum_subs'), max('sum_subs')]).show()

+--------------------+-------------+-------------+
|       avg(sum_subs)|min(sum_subs)|max(sum_subs)|
+--------------------+-------------+-------------+
|1.311021117062158E-4|            0|            9|
+--------------------+-------------+-------------+



In [156]:
cluster_1.select([mean('sum_discont'), min('sum_discont'), max('sum_discont')]).show()

+-------------------+----------------+----------------+
|   avg(sum_discont)|min(sum_discont)|max(sum_discont)|
+-------------------+----------------+----------------+
|0.06770882822375883|               0|               7|
+-------------------+----------------+----------------+

